In [1]:
# If the libraries are not yet installed, they can be installed in this notebook using commands similar to the below
# %conda install numpy
# %conda install pandas
# %conda install matplotlib
# %conda install scikit-learn
# %conda install -c conda-forge lightgbm 
# %conda install -c conda-forge swifter
# %conda install -c conda-forge bayesian-optimization 
# %conda install -c conda-forge scipy
# %conda install joblib
# %conda install tdqm

# Something like the following may also work if the above does not
# import sys
# !conda install --yes --prefix {sys.prefix} numpy
# !conda install --yes --prefix {sys.prefix} pandas
# !conda install --yes --prefix {sys.prefix} scikit-learn
# !conda install -c conda-forge --yes --prefix {sys.prefix} lightgbm
# !conda install -c conda-forge --yes --prefix {sys.prefix} swifter
# !conda install -c conda-forge --yes --prefix {sys.prefix} bayesian-optimization 
# !conda install -c conda-forge --yes --prefix {sys.prefix} scipy 
# !conda install --yes --prefix {sys.prefix} joblib
# !conda install --yes --prefix {sys.prefix} tdqm

# To install a specific version, add the version to the install command
# E.g., %conda install numpy=1.20.3

# If all else fails, use pip or follow additional advice such as found at
# https://jakevdp.github.io/blog/2017/12/05/installing-python-packages-from-jupyter/

# If your plan to use pip (especially if you are not working within a specified conda environment), 
# the pip commands might look like:
# pip install numpy
# pip install pandas
# pip install scikit-learn
# pip install lightgbm
# pip install swifter
# pip install bayesian-optimization 
# pip install scipy
# pip install joblib
# pip install tdqm

# To install a specific version, add the version to the pip install command
# E.g., pip install numpy==1.20.3

In [2]:
import time
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
import itertools
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import glob
from lightgbm import LGBMRegressor
import random
from sklearn.model_selection import ParameterSampler
import scipy
import gc
from joblib import Parallel, delayed
import contextlib
import joblib
from tqdm import tqdm
from bayes_opt import BayesianOptimization
import os

In [3]:
# Set random seed for reproducibility
np.random.seed(54321)
random.seed(54321)

In [4]:
# Create directories to store results
os.makedirs("Results/Univariate/LightGBM Default/", exist_ok=True)
os.makedirs("Results/Univariate/LightGBM Bayes/", exist_ok=True)

# Read in Data and Prepare for Modeling

In [5]:
# Create an empty list to hold the dataframes of highways england data
england_df_list = list()

# Loop through the files, sorted in alphabetical order
# Read them into a df, make sure they are sorted by timestamp, and append to the list
for fname in sorted(glob.glob("Data/Processed/Highways_England/*.csv")):
    print("Reading {}".format(fname))
    df = pd.read_csv(fname) #, parse_dates=['timestamp'], index_col=['timestamp'])
    df = df.sort_values(by="timestamp")
    england_df_list.append(df)

Reading Data/Processed/Highways_England/A11-6310-1_Southbound_2019_Processed.csv
Reading Data/Processed/Highways_England/A11-6312-2_Northbound_2019_Processed.csv
Reading Data/Processed/Highways_England/A14-1107A_Eastbound_2019_Processed.csv
Reading Data/Processed/Highways_England/A14-1144B_Westbound_2019_Processed.csv
Reading Data/Processed/Highways_England/A1M-9842B_Southbound_2019_Processed.csv
Reading Data/Processed/Highways_England/A1M-9847a_Northbound_2019_Processed.csv
Reading Data/Processed/Highways_England/A46-7636-1_Northbound_2019_Processed.csv
Reading Data/Processed/Highways_England/A46-7636-2_Southbound_2019_Processed.csv
Reading Data/Processed/Highways_England/A47-6337-1_Westbound_2019_Processed.csv
Reading Data/Processed/Highways_England/A47-6337-2_Eastbound_2019_Processed.csv
Reading Data/Processed/Highways_England/A5-6847-2_Southbound_2019_Processed.csv
Reading Data/Processed/Highways_England/A5-7572-1-Northbound_2019_Processed.csv
Reading Data/Processed/Highways_Englan

In [6]:
# Follow the same process in this cell and the next as was done above, just for other highway systems
portland_df_list = list()

for fname in sorted(glob.glob("Data/Processed/Portland/*.csv")):
    print("Reading {}".format(fname))
    df = pd.read_csv(fname) #, parse_dates=['timestamp'], index_col=['timestamp'])
    df = df.sort_values(by="timestamp")
    portland_df_list.append(df)

Reading Data/Processed/Portland/I205-101068_Northbound_2019_Processed.csv
Reading Data/Processed/Portland/I205-101073_Southbound_2019_Processed.csv
Reading Data/Processed/Portland/I405-100395_Southbound_2019_Processed.csv
Reading Data/Processed/Portland/I405-100527_Northbound_2019_Processed.csv
Reading Data/Processed/Portland/I5-100688_Southbound_2019_Processed.csv
Reading Data/Processed/Portland/I5-100703_Northbound_2019_Processed.csv
Reading Data/Processed/Portland/I84-101108_Eastbound_2019_Processed.csv
Reading Data/Processed/Portland/I84-101161_Westbound_2019_Processed.csv
Reading Data/Processed/Portland/OR217-100300_Southbound_2019_Processed.csv
Reading Data/Processed/Portland/OR217-100314_Northbound_2019_Processed.csv
Reading Data/Processed/Portland/R2 Delta Hwy-101745_Northbound_2019_Processed.csv
Reading Data/Processed/Portland/R2 OR18-102111_Westbound_2019_Processed.csv
Reading Data/Processed/Portland/R2 OR18-102113_Eastbound_2019_Processed.csv
Reading Data/Processed/Portland/

In [7]:
utah_df_list = list()

for fname in sorted(glob.glob("Data/Processed/Utah/*.csv")):
    print("Reading {}".format(fname))
    df = pd.read_csv(fname) #, parse_dates=['timestamp'], index_col=['timestamp'])
    df = df.sort_values(by="timestamp")
    utah_df_list.append(df)

Reading Data/Processed/Utah/I15-3103178_Southbound_2019_Processed.csv
Reading Data/Processed/Utah/I15-749_Northbound_2019_Processed.csv
Reading Data/Processed/Utah/I215-134_Counterclockwise_2019_Processed.csv
Reading Data/Processed/Utah/I215-31_Clockwise_2019_Processed.csv
Reading Data/Processed/Utah/I70-3103400_Westbound_2019_Processed.csv
Reading Data/Processed/Utah/I70-3103401_Eastbound_2019_Processed.csv
Reading Data/Processed/Utah/I80-600_Eastbound_2019_Processed.csv
Reading Data/Processed/Utah/I80-667_Westbound_2019_Processed.csv
Reading Data/Processed/Utah/I84-451_Eastbound_2019_Processed.csv
Reading Data/Processed/Utah/I84-482_Westbound_2019_Processed.csv
Reading Data/Processed/Utah/LegacyParkway-810_Northbound_2019_Processed.csv
Reading Data/Processed/Utah/LegacyParkway-890_Southbound_2019_Processed.csv
Reading Data/Processed/Utah/US189-260_Westbound_2019_Processed.csv
Reading Data/Processed/Utah/US189-470_Eastbound_2019_Processed.csv
Reading Data/Processed/Utah/US40-634_Westb

In [8]:
# Append all df lists together into one
total_df_list = england_df_list + portland_df_list + utah_df_list

In [9]:
# Read in the start and end points csv, and subtract 1 to deal with index differences between R and python
start_end = pd.read_csv("start_end_points.csv")
start_end["start"] = start_end["start"] - 1
start_end["end"] = start_end["end"]

In [10]:
# Create an empty list to hold the subset data frames (those with only 12 weeks of data per highway)
subset_df_list = list()

In [11]:
# For each df in our original total df list
for idx, df in enumerate(total_df_list):
        
    # Filter the timeframe based on the start_end_points csv files
    subset_df = df.iloc[start_end.iloc[idx,0]:start_end.iloc[idx,1], ]\
    .reset_index(drop=True).reset_index(drop=False)\
    .rename(columns={"index":"rn"})
    
    # Create a new field called train_val_test to differentiate each set of data
    subset_df["train_val_test"] = np.where(subset_df["rn"]<(96*7*8),
                                           "train",
                                           np.where(subset_df["rn"]<(96*7*10),
                                                    "val",
                                                    "test"
                                                   )
                                       )
    
    # Append to list
    subset_df_list.append(subset_df)

In [12]:
# Create a list of df's with only fields we need

# Initialize empty list
model_df_list = list()

# For df in subset list
for df in subset_df_list:
       
    # Extract the timestamp, the volume, and the train_val_test assignment
    model_df = df[['timestamp', 'total_volume', "train_val_test"]]\
    .rename(columns={'timestamp':'start', 'total_volume':'target'})
    
    # Append this df to the new list
    model_df_list.append(model_df)

## Create Lag Emebedded Matrices for each TS

In [13]:
# Set the final lag value to be used for all lag embedding
lag_n = 840

In [14]:
# Create an empty list to save lag embedded data into
lag_embed_df_list = list()

# For each data frame
for df in model_df_list:
    for n in range(1, (lag_n+1)):
        # For each lag level, up to lag_n + 1 (we add 1 to preserve the target value correctly)
        # Create a new column called target-n
        name = f"target-{n}"
        # Save the target shifted n values into this column
        df[name] = df['target'].shift(n)
    # Append the lag embedded df to the list
    lag_embed_df_list.append(df)

/tmp/ipykernel_10922/1739749478.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[name] = df['target'].shift(n)


In [15]:
# Split the lag embedded list into train, val, and test lists

# First, initialize empty lists for each train, val, and test
train_df_list = list()
val_df_list = list()
test_df_list = list()

# For each df in our list
for i in range(len(lag_embed_df_list)):
    
    # Create a copy of just the data frame of interest
    df = lag_embed_df_list[i].copy()
    # Add a field to it for ts_index, this is for joining with cluster data later and is equal to i+1 due to 
    # differences in indexing between R and Python
    df['ts_index'] = i + 1
    
    # Subset into train, val, and test df's based on the train_val_test_field
    train_df = df.query("train_val_test == 'train'").copy()
    val_df = df.query("train_val_test=='val'").copy()
    test_df = df.query("train_val_test=='test'").copy()
   
    # Append to appropriate lists
    train_df_list.append(train_df)
    val_df_list.append(val_df)
    test_df_list.append(test_df)

In [16]:
# Concat all dfs from the lists together to create one full train, val, and test df
train_df_full = pd.concat(train_df_list)
val_df_full = pd.concat(val_df_list)
test_df_full = pd.concat(test_df_list)

In [17]:
# Drop unneeded columns
train_df_full.drop(columns=['start', 'train_val_test'], inplace=True)
val_df_full.drop(columns=['start', 'train_val_test'], inplace=True)
test_df_full.drop(columns=['start', 'train_val_test'], inplace=True)

In [18]:
# Append the training and validation data together for later use
train_val_full = train_df_full.append(val_df_full)

# Model Each Time Series Using Default LightGBM Parameters

In [19]:
# Define a function to train a light gbm model
def train_local_lgbm(m, data, n):
    """Function which takes a time series index m, a training data frame, and a lag value n and trains a model"""
    
    # Create y and X data frames from the trianing data for ts_index m and lag embedding n 
    y_train_sub = data.query("ts_index==@m").iloc[n:,0]
    X_train_sub = data.query("ts_index==@m").iloc[n:,0:(n+1)].iloc[:,1:]
    
    # Create the model object and fit it
    mod_sub = LGBMRegressor(boosting_type='goss', random_state=54321)
    mod_sub.fit(X_train_sub, y_train_sub)
    
    # Return the fitted model
    return mod_sub

In [20]:
# Code for progress bar:
# https://stackoverflow.com/questions/24983493/tracking-progress-of-joblib-parallel-execution
# This allows us to print a progress bar while running parallel loops using joblib 

@contextlib.contextmanager
def tqdm_joblib(tqdm_object):
    """Context manager to patch joblib to report into tqdm progress bar given as argument"""
    class TqdmBatchCompletionCallback(joblib.parallel.BatchCompletionCallBack):
        def __call__(self, *args, **kwargs):
            tqdm_object.update(n=self.batch_size)
            return super().__call__(*args, **kwargs)

    old_batch_callback = joblib.parallel.BatchCompletionCallBack
    joblib.parallel.BatchCompletionCallBack = TqdmBatchCompletionCallback
    try:
        yield tqdm_object
    finally:
        joblib.parallel.BatchCompletionCallBack = old_batch_callback
        tqdm_object.close()

In [21]:
# Loop through the list of time series in parallel and train a LGBM model for each, saving models to a list
with tqdm_joblib(tqdm(desc="Local Models - Default Params", total=76)) as progress_bar:
    results = Parallel(n_jobs=4)(delayed(train_local_lgbm)(i, train_val_full, lag_n) for i in range(1, 77))

Local Models - Default Params: 100%|████████████| 76/76 [01:38<00:00,  1.29s/it]


In [22]:
# Save the trained models in the list to files
for i in range(76):
    fname = f"Results/Univariate/LightGBM Default/model_{i}"
    joblib.dump(results[i], fname)

In [23]:
# Load the trained models from files

# Create an empty list
results = list()

# For each time series, load the model for that time series and append it into the list
for i in range(76):
    fname = f"Results/Univariate/LightGBM Default/model_{i}"
    results.append(joblib.load(fname))

In [24]:
# Function to make predicitons on the test data
def predict_test_lgbm(model, m, data, n):
    """Function which takes in a trained model, time series index, test data frame, and lag embedding
    and returns a dictionary of model test prediction performance"""
    
    # Create y and X data frames for the given ts index and lag embedding
    y_test_sub = data.query("ts_index==@m").iloc[:,0]
    X_test_sub = data.query("ts_index==@m").iloc[:,1:(n+1)]
    
    # Predict on the test data
    test_preds_sub = model.predict(X_test_sub)
    
    # Compute the mean of the true test data as well as the mae and rmse of the predictions
    test_mean = np.mean(y_test_sub)
    test_mae = mean_absolute_error(y_test_sub, test_preds_sub)
    test_rmse = mean_squared_error(y_test_sub, test_preds_sub, squared=False)
    
    # Save the mean, mae, and rmse into a dictionary
    pred_perf = {"mean": test_mean, "mae": test_mae, "rmse": test_rmse}
    
    # Return the dictionary
    return pred_perf

In [25]:
# Loop through all time series and save the prediction performance dictionaries for each to a list
with tqdm_joblib(tqdm(desc="Local Models - Test Preds", total=76)) as progress_bar:
    test_results = Parallel(n_jobs=4)(delayed(predict_test_lgbm)(results[i], i+1, test_df_full, lag_n) for i in range(76))

Local Models - Test Preds: 100%|████████████████| 76/76 [00:06<00:00, 11.01it/s]


In [26]:
# Create a data frame from that list of dictionaries
local_model_test_perf = pd.DataFrame(test_results)

In [27]:
local_model_test_perf.head()

,mean,mae,rmse
0,259.136161,18.247170,24.858248
1,195.970982,18.816907,27.354514
2,207.250000,16.952075,23.865056
3,395.383929,26.783249,40.812835
4,445.119048,36.697384,57.103106


In [28]:
# Add scaled metrics to that data frame
local_model_test_perf['nrmse'] = local_model_test_perf['rmse']/local_model_test_perf['mean']
local_model_test_perf['smae'] = local_model_test_perf['mae']/local_model_test_perf['mean']

In [29]:
# Print the means of the prediction performance metrics
local_model_test_perf.mean()

mean     265.435072
mae       21.956678
rmse      32.997245
nrmse      0.145757
smae       0.098317
dtype: float64

In [30]:
# Load in the saved models (starting from a different time, so loading instead of re-creating)
base_mod_ls = list()

for i in range(76):
    fname = f"Results/Univariate/LightGBM Default/model_{i}"
    base_mod_ls.append(joblib.load(fname))

In [31]:
# Create an empty dictionary to store residuals in
base_mod_resiudal_dict = dict()

# Loop through each time series index
for i in range(1, 77):
    # Extract the y and X train for each index
    y_train_sub = train_val_full.query("ts_index==@i").iloc[:,0:(lag_n+1)].dropna().iloc[:,0].copy()
    X_train_sub = train_val_full.query("ts_index==@i").iloc[:,0:(lag_n+1)].dropna().iloc[:,1:].copy()
    
    # Make predictions on the training data
    train_preds_sub = base_mod_ls[i-1].predict(X_train_sub)
    
    # Compute residuals and convery to list
    res_sub = (y_train_sub - train_preds_sub).to_list()
    
    # Save that list into the dictionary, with key equal to the ts index
    base_mod_resiudal_dict[i] = res_sub

In [32]:
# Create an empty dictionary for test preds
base_mod_test_preds = dict()

# For each ts index
for i in range(1,77):
    # Create the X data frame from the test data for that ts
    X_test_sub = test_df_full.query("ts_index==@i").iloc[:,1:(lag_n+1)]
    
    # Predict on the test data
    test_pred_sub = base_mod_ls[i-1].predict(X_test_sub)
    
    # Save those preds to the dictionary
    base_mod_test_preds[i] = test_pred_sub

In [33]:
# Set n_boot to 1000
n_boot = 1000

# Create an empty df to store pred intervals
pred_int_df = pd.DataFrame()

# Set random seed for reproducibility
np.random.seed(54321)
random.seed(54321)

# Loop through all ts indexes
for i in range(1,77):

    # Print i to monitor progress
    print(i)
    
    # Get the true/target value of y for the test data for that ts
    y_test_sub = test_df_full.query("ts_index==@i").iloc[:,0]
    
    # Create empty lists to store PIs in
    percent_95_lo_ls = list()
    percent_95_hi_ls = list()
    percent_80_lo_ls = list()
    percent_80_hi_ls = list()
    
    # Loop through the number of observations in the test set
    for j in range(1344):
        
        # Grab the test pred for the given ts index and observation number
        pred = base_mod_test_preds[i][j]
        # Sample n_boot times from the appropriate model's residuals
        resid_boot = np.random.choice(base_mod_resiudal_dict[i], size=n_boot, replace=True)
        # Add the test pred to the residuals
        resid_preds = pred+resid_boot
        
        # Compute the percentiles of resid_preds for the 95% PI
        percent_95_lo = np.percentile(resid_preds, 2.5)
        percent_95_hi = np.percentile(resid_preds, 97.5)
        percent_95_lo_ls.append(percent_95_lo)
        percent_95_hi_ls.append(percent_95_hi)
        
        # Compute the percentiles of resid_preds for the 80% PI
        percent_80_lo = np.percentile(resid_preds, 10)
        percent_80_hi = np.percentile(resid_preds, 90)
        percent_80_lo_ls.append(percent_80_lo)
        percent_80_hi_ls.append(percent_80_hi)
    
    # Create a temp data frame with the ts_index, true values, and PIs
    pred_int_df_sub = pd.DataFrame({"ts_index": i,
                                    "actual": y_test_sub,
                                    "pct_95_lo": percent_95_lo_ls,
                                    "pct_95_hi": percent_95_hi_ls,                                   
                                    "pct_80_lo": percent_80_lo_ls,
                                    "pct_80_hi": percent_80_hi_ls
                                   })
    
    # Append to the full data frame
    pred_int_df = pred_int_df.append(pred_int_df_sub)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76


In [34]:
# Sanity check
pred_int_df.head()

,ts_index,actual,pct_95_lo,pct_95_hi,pct_80_lo,pct_80_hi
6720,1,320.0,310.643649,366.575901,318.977586,356.605693
6721,1,339.0,310.294098,363.459622,319.234061,354.396589
6722,1,349.0,336.929812,387.131197,344.183587,378.530008
6723,1,343.0,327.587767,384.485951,337.377908,372.680660
6724,1,343.0,322.840252,377.061014,331.064423,366.445963


In [35]:
# Sanity check
pred_int_df.ts_index.unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75, 76])

In [36]:
# Sanity check
len(pred_int_df.ts_index.unique())

76

In [37]:
# Create a function to compute the interval score
def interval_score(true_values, lower, upper, interval_range):
    """ Function which takes in the true values, the upper and lower bounds of PIs, and the PI level (e.g., 90%)
        and from these inputs, computes the interval score for each prediction
    """
    
    # Compute alpha from the interval range
    alpha = 1-interval_range
    
    # Save the upper, lower, and true_values as numpy arrays for computation purposes
    upper = np.array(upper)
    lower = np.array(lower)
    true_values = np.array(true_values)
    
    # Compute the lower component of the interval score - just a boolean for true below interval
    def lower_ind(true,low):
        if true<low:
            return 1
        else:
            return 0
        
    # Computer the upper component of the interval score - similar boolean for true above interval
    def upper_ind(true,up):
        if true>up:
            return 1
        else:
            return 0
        
    # Computer the actual score for each obsveration - formula here: https://epiforecasts.io/scoringutils/reference/interval_score.html
    scores = (upper-lower) + (2/alpha)*(lower-true_values)*(lower > true_values) + (2/alpha)*(true_values-upper)*(true_values > upper)
    
    # Return the scores array
    return scores

In [38]:
# Compute the 80% and 95% PI scores using the above function
pred_int_df['int_80_score'] = interval_score(pred_int_df['actual'], 
                                             pred_int_df['pct_80_lo'], 
                                             pred_int_df['pct_80_hi'],
                                             0.8
                                            )

pred_int_df['int_95_score'] = interval_score(pred_int_df['actual'], 
                                             pred_int_df['pct_95_lo'], 
                                             pred_int_df['pct_95_hi'],
                                             0.95
                                            )

In [39]:
# Print the mean of the PI scores
pred_int_df.mean()

ts_index         38.500000
actual          265.435072
pct_95_lo       238.678292
pct_95_hi       293.148442
pct_80_lo       248.090553
pct_80_hi       283.636933
int_80_score    137.061209
int_95_score    330.857352
dtype: float64

In [40]:
# Group by time series index and compute mean interval scores and actual mean
pred_int_df_grouped = pred_int_df.groupby("ts_index")\
.agg({"int_95_score":"mean", "int_80_score":"mean", "actual":"mean"}).reset_index()

# Compute scaled interval scores
pred_int_df_grouped['int_95_score_scaled'] = pred_int_df_grouped['int_95_score']/pred_int_df_grouped['actual']
pred_int_df_grouped['int_80_score_scaled'] = pred_int_df_grouped['int_80_score']/pred_int_df_grouped['actual']

In [41]:
# Print the average scaled interval scores
pred_int_df_grouped[['int_80_score_scaled', 'int_95_score_scaled']].mean()

int_80_score_scaled    0.611034
int_95_score_scaled    1.470294
dtype: float64

In [42]:
# Save the PI data frame to a csv file
pred_int_df.to_csv("Results/Univariate/LightGBM Default/prediction_intervals.csv", index=False)

# Local Models with Bayesian Optimization

In [43]:
# Define function to perform Bayesian optimization
def optimize_lgbm_w_bayes(i, lag_n, train_df, val_df):
    """Function which takes in a time series index i, a lag embedding lag_n, and a train and validation
    data frame and which returns the best model params found using bayesian optimization"""

    # Subset the input train data into X and y data frames for the provided index i
    y_train_bayes = train_df.query("ts_index==@i").iloc[:,0:(lag_n+1)].dropna().iloc[:,0].copy()
    X_train_bayes = train_df.query("ts_index==@i").iloc[:,0:(lag_n+1)].dropna().iloc[:,1:].copy()
    
    # Get the validation data for the provided ts_index i
    val_df_bayes = val_df.query("ts_index==@i").copy()
   
    # Set up space of lgbm params to explore
    bayes_param_ss = {
    "n_estimators": (100, 1000),
    "max_depth": (2, 25),
    "lambda_l1": (0, 1),
    "lambda_l2": (0, 1),
    "num_leaves": (10, 150),
    "colsample_bytree": (0.1, 1),
    "learning_rate": (0.00001, 0.5)
    }
    

    # Define a function to compute validation set predictions
    def val_predict(model, val_df):
        """Function takes in a trained model and validation data and returns normalized rmse for preds on the 
        validation data"""
        
        # Subset the validation data frame into y and X data frames
        y_val_sub = val_df.iloc[:,0]
        X_val_sub = val_df.iloc[:,1:(lag_n+1)]
        # Compute the mean of the true y values
        val_mean_sub = np.mean(y_val_sub)

        # Make model predictions
        val_preds_sub = model.predict(X_val_sub)

        # Compute rmse on the predictions, and then divide by the mean to get nrmse
        val_rmse_sub = mean_squared_error(y_val_sub, val_preds_sub, squared=False)
        val_nrmse_sub = val_rmse_sub/val_mean_sub

        # Return normalized rmse
        return val_nrmse_sub
    
    
    # Define function to perform the bayesian optimization
    def lgbm_eval_for_bayes(n_estimators,
                        max_depth,
                        lambda_l1, 
                        lambda_l2,
                        num_leaves,
                        colsample_bytree,
                        learning_rate
                       ):
        """Function which takes in parameter values as inputs and returns a value to be maximized by the
        Bayesian optimizer. In this case, we return -1*validation_nrmse as this allows us to minimize the
        validation nrmse"""
        
        # Set the proper boosting type
        params = {"boosting_type": "goss"
                 }
        
        # Set the params dictionary to include all input params
        # For n_estimators, max_depth, and num_leaves, round and cast as int - this is what the lgbm model requires
        params["n_estimators"] = int(round(n_estimators))
        params["max_depth"] = int(round(max_depth))
        params["reg_alpha"] = max(lambda_l1, 0)
        params["reg_lambda"] = max(lambda_l2, 0)
        params["num_leaves"] = int(round(num_leaves))
        params["colsample_bytree"] = colsample_bytree
        params["learning_rate"] = learning_rate

        # Create the model object, setting a constant random_state for reproducibility, and fit the model
        mod = LGBMRegressor(**params, random_state=54321)  
        mod.fit(X_train_bayes, y_train_bayes)

        # Compute the performance on the validation data and multiple by -1
        val_perf = -1*np.mean(val_predict(mod, val_df_bayes))

        # Return the negative validation nrmse
        return val_perf

    # Create an optimizer object, again setting random_state
    optimizer = BayesianOptimization(lgbm_eval_for_bayes,
                                     bayes_param_ss,
                                     random_state=54321)
    # Run the optimizer with 5 random initialization points and 25 further iterations
    optimizer.maximize(init_points=5, n_iter=25)
    
    # Return the best params found by the optimizer
    return optimizer.max['params']

In [44]:
# In parallel, for all the time series in our list, 
# loop through and run the Bayesian optimzer. Save the params for each model to a list
with tqdm_joblib(tqdm(desc="Local Models - Bayesian Optim", total=76)) as progress_bar:
    local_bayes_results = Parallel(n_jobs=6)(delayed(optimize_lgbm_w_bayes)(i, 
                                                                            lag_n, 
                                                                            train_df_full,
                                                                            val_df_full
                                                                           ) for i in range(1, 77))

Local Models - Bayesian Optim:   4%|▍         | 3/76 [10:22<3:23:35, 167.33s/it]

|   iter    |  target   | colsam... | lambda_l1 | lambda_l2 | learni... | max_depth | n_esti... | num_le... |
-------------------------------------------------------------------------------------------------------------
|  1        | -0.1033   |  0.9205   |  0.6238   |  0.7918   |  0.2149   |  14.49    |  472.1    |  21.99    |
|  2        | -0.1165   |  0.7999   |  0.4889   |  0.05053  |  0.2692   |  2.954    |  827.3    |  149.3    |
|  3        | -0.129    |  0.6748   |  0.9459   |  0.7367   |  0.411    |  4.633    |  761.5    |  125.0    |
|  4        | -0.1388   |  0.1665   |  0.2079   |  0.8057   |  0.4572   |  5.424    |  962.3    |  62.04    |
|  5        | -0.1087   |  0.7467   |  0.1039   |  0.8769   |  0.2918   |  17.67    |  897.5    |  35.06    |
|  6        | -0.09812  |  0.1328   |  0.1594   |  0.3429   |  0.03426  |  3.006    |  829.8    |  148.7    |
|  7        | -0.09998  |  0.1823   |  0.2334   |  0.3629   |  0.06587  |  17.26    |  859.2    |  141.3    |
|  8      

Local Models - Bayesian Optim:   7%|▋          | 5/76 [11:48<1:50:47, 93.62s/it]

|   iter    |  target   | colsam... | lambda_l1 | lambda_l2 | learni... | max_depth | n_esti... | num_le... |
-------------------------------------------------------------------------------------------------------------
|  1        | -0.1025   |  0.9205   |  0.6238   |  0.7918   |  0.2149   |  14.49    |  472.1    |  21.99    |
|  2        | -0.1085   |  0.7999   |  0.4889   |  0.05053  |  0.2692   |  2.954    |  827.3    |  149.3    |
|  3        | -0.1247   |  0.6748   |  0.9459   |  0.7367   |  0.411    |  4.633    |  761.5    |  125.0    |
|  4        | -0.15     |  0.1665   |  0.2079   |  0.8057   |  0.4572   |  5.424    |  962.3    |  62.04    |
|  5        | -0.1131   |  0.7467   |  0.1039   |  0.8769   |  0.2918   |  17.67    |  897.5    |  35.06    |
|  6        | -0.09398  |  0.1328   |  0.1594   |  0.3429   |  0.03426  |  3.006    |  829.8    |  148.7    |
|  7        | -0.102    |  0.4892   |  0.1292   |  0.7004   |  0.1635   |  6.494    |  832.8    |  149.1    |
|  8      

|   iter    |  target   | colsam... | lambda_l1 | lambda_l2 | learni... | max_depth | n_esti... | num_le... |
-------------------------------------------------------------------------------------------------------------
|  1        | -0.1517   |  0.9205   |  0.6238   |  0.7918   |  0.2149   |  14.49    |  472.1    |  21.99    |
|  2        | -0.1652   |  0.7999   |  0.4889   |  0.05053  |  0.2692   |  2.954    |  827.3    |  149.3    |
|  3        | -0.1778   |  0.6748   |  0.9459   |  0.7367   |  0.411    |  4.633    |  761.5    |  125.0    |
|  4        | -0.2198   |  0.1665   |  0.2079   |  0.8057   |  0.4572   |  5.424    |  962.3    |  62.04    |
|  5        | -0.1592   |  0.7467   |  0.1039   |  0.8769   |  0.2918   |  17.67    |  897.5    |  35.06    |
|  6        | -0.1488   |  0.1328   |  0.1594   |  0.3429   |  0.03426  |  3.006    |  829.8    |  148.7    |
|  7        | -0.1513   |  0.4892   |  0.1292   |  0.7004   |  0.1635   |  6.494    |  832.8    |  149.1    |
|  8      

...

Local Models - Bayesian Optim:  97%|█████████▋| 74/76 [2:03:44<02:17, 68.88s/it]

|  5        | -0.2171   |  0.7467   |  0.1039   |  0.8769   |  0.2918   |  17.67    |  897.5    |  35.06    |
|  6        | -0.197    |  0.1328   |  0.1594   |  0.3429   |  0.03426  |  3.006    |  829.8    |  148.7    |
|  7        | -0.1944   |  0.4892   |  0.1292   |  0.7004   |  0.1635   |  6.494    |  832.8    |  149.1    |
|  8        | -0.1939   |  0.8211   |  0.9199   |  0.748    |  0.1659   |  2.619    |  839.8    |  144.5    |
|  9        | -0.2539   |  0.3894   |  0.0252   |  0.3812   |  0.418    |  4.29     |  839.3    |  132.4    |
|  10       | -0.2573   |  0.9397   |  0.8542   |  0.2827   |  0.4121   |  2.9      |  846.3    |  148.8    |
|  11       | -0.2175   |  0.2227   |  0.9983   |  0.6203   |  0.3183   |  7.491    |  834.4    |  143.1    |
|  12       | -0.202    |  0.3732   |  0.832    |  0.6506   |  0.1811   |  4.793    |  837.6    |  149.0    |
|  13       | -0.1942   |  0.3652   |  0.1928   |  0.8821   |  0.09737  |  10.25    |  473.0    |  15.08    |
|  14     

Local Models - Bayesian Optim:  99%|█████████▊| 75/76 [2:05:24<01:18, 78.25s/it]

|  4        | -0.3186   |  0.1665   |  0.2079   |  0.8057   |  0.4572   |  5.424    |  962.3    |  62.04    |
|  5        | -0.2319   |  0.7467   |  0.1039   |  0.8769   |  0.2918   |  17.67    |  897.5    |  35.06    |
|  6        | -0.2218   |  0.1328   |  0.1594   |  0.3429   |  0.03426  |  3.006    |  829.8    |  148.7    |
|  7        | -0.2518   |  0.4405   |  0.1374   |  0.64     |  0.3496   |  2.775    |  837.2    |  144.8    |
|  8        | -0.2449   |  0.5195   |  0.9015   |  0.7068   |  0.328    |  8.482    |  470.1    |  22.5     |
|  9        | -0.2878   |  0.2672   |  0.6743   |  0.8705   |  0.4451   |  16.88    |  479.3    |  21.02    |
|  10       | -0.2482   |  0.5874   |  0.748    |  0.8152   |  0.278    |  12.53    |  465.6    |  20.65    |
|  11       | -0.2344   |  0.225    |  0.9094   |  0.07102  |  0.01027  |  7.201    |  833.4    |  149.4    |
|  12       | -0.2244   |  0.6713   |  0.4853   |  0.7558   |  0.1096   |  16.73    |  471.9    |  27.24    |
|  13     

Local Models - Bayesian Optim: 100%|█████████| 76/76 [2:06:42<00:00, 100.03s/it]


In [45]:
# Create a data frame of model params and save to a file in case it is needed later
local_bayes_param_df = pd.DataFrame(local_bayes_results)
local_bayes_param_df.to_csv("Results/Univariate/LightGBM Bayes/best_params_bayes.csv", index=False)

In [46]:
# Define function to train a final model
def train_best_lgbm_local(m, data, param_ls, lag_n):
    """Function which takes inputs: m, the ts index, data, the full training_validation data frame,
    param_ls, the list of params from which to choose, and lag_n, the lag embedding of the data,
    and which returns a fitted model"""
    
    # Subset the data to the appropriate ts index and lag and split into X and y data frames
    y_train_sub = data.query("ts_index==@m").iloc[:,0:(lag_n+1)].dropna().iloc[:,0].copy()
    X_train_sub = data.query("ts_index==@m").iloc[:,0:(lag_n+1)].dropna().iloc[:,1:].copy()
    
    # Extract the params for this model
    params = param_ls[m-1]
    # Round and cast to int the params which must be integers
    params['n_estimators'] = int(round(params['n_estimators']))
    params['num_leaves'] = int(round(params['num_leaves']))
    params['max_depth'] = int(round(params['max_depth']))
    
    # Create and fit the model object
    mod_sub = LGBMRegressor(boosting_type="goss", **params, random_state=54321)
    mod_sub.fit(X_train_sub, y_train_sub)

    # Return the fitted model
    return mod_sub

In [47]:
# In parallel, lopo through the list of ts indexes, model params, etc., and train the best local model for each 
# ts in our data set. Save to a list
with tqdm_joblib(tqdm(desc="Local Models - Bayesian Optim", total=76)) as progress_bar:
    final_local_bayes_models = Parallel(n_jobs=4)(delayed(train_best_lgbm_local)(i, 
                                                                                 train_val_full, 
                                                                                 local_bayes_results,
                                                                                 lag_n
                                                                                ) for i in range(1, 77))

Local Models - Bayesian Optim:   0%|                     | 0/76 [00:00<?, ?it/s]

|  19       | -0.09926  |  0.4944   |  0.3307   |  0.4453   |  0.03993  |  16.18    |  472.7    |  25.08    |
|  20       | -0.1078   |  0.5192   |  0.1595   |  0.8201   |  0.1304   |  16.74    |  468.9    |  25.39    |
|  21       | -0.1566   |  0.9372   |  0.5743   |  0.2607   |  0.4664   |  17.07    |  467.5    |  23.06    |
|  22       | -0.09978  |  0.3324   |  0.5674   |  0.3411   |  0.01824  |  2.645    |  832.6    |  149.3    |
|  23       | -0.1043   |  0.6098   |  0.4088   |  0.2748   |  0.1032   |  17.53    |  471.8    |  26.53    |
|  24       | -0.1025   |  0.6447   |  0.8453   |  0.6332   |  0.06772  |  10.63    |  474.3    |  26.43    |
|  25       | -0.1375   |  0.6251   |  0.3562   |  0.2435   |  0.3525   |  8.113    |  476.6    |  27.67    |
|  26       | -0.1073   |  0.7345   |  0.7755   |  0.5436   |  0.1235   |  19.03    |  472.8    |  29.46    |
|  27       | -0.106    |  0.6403   |  0.5419   |  0.01567  |  0.09898  |  10.34    |  470.2    |  28.56    |
|  28     

Local Models - Bayesian Optim:  99%|███████████▊| 75/76 [03:57<00:04,  4.04s/it]

[LightGBM] [Warning] lambda_l1 is set=0.8599685060636071, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.8599685060636071
[LightGBM] [Warning] lambda_l2 is set=0.6944617913953867, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.6944617913953867


Local Models - Bayesian Optim: 100%|████████████| 76/76 [04:01<00:00,  3.18s/it]


In [48]:
# Write the trained models to files for future use if needed
for n in range(76):
    fname = f"Results/Univariate/LightGBM Bayes/model_{n}"
    joblib.dump(final_local_bayes_models[n], fname)

In [49]:
# Load the models from files

# Create an empty list
final_local_bayes_models = list()

# Loop through all ts_index, load the model with joblib, and append to the list
for n in range(76):
    fname = f"Results/Univariate/LightGBM Bayes/model_{n}"
    final_local_bayes_models.append(joblib.load(fname))

In [50]:
# In parallel, loop through all of our trained models and test sets and compute prediction performance
with tqdm_joblib(tqdm(desc="Local Models Bayes - Test Preds", total=76)) as progress_bar:
    test_results_bayes = Parallel(n_jobs=4)(delayed(predict_test_lgbm)(final_local_bayes_models[i], 
                                                                       i+1, 
                                                                       test_df_full, 
                                                                       lag_n) for i in range(76))

Local Models Bayes - Test Preds: 100%|██████████| 76/76 [00:09<00:00,  7.85it/s]


In [51]:
# Save the test set performance list into a df
test_results_bayes_df = pd.DataFrame(test_results_bayes)

In [52]:
# Add normalized metrics to the df
test_results_bayes_df['nrmse'] = test_results_bayes_df['rmse']/test_results_bayes_df['mean']
test_results_bayes_df['smae'] = test_results_bayes_df['mae']/test_results_bayes_df['mean']

In [53]:
# Print the means of the normalized metrics
test_results_bayes_df.mean()

mean     265.435072
mae       22.206809
rmse      33.806245
nrmse      0.145439
smae       0.097278
dtype: float64

In [54]:
# Load the previously saved models from files into a list
bayes_mod_ls = list()

for i in range(76):
    fname = f"Results/Univariate/LightGBM Bayes/model_{i}"
    bayes_mod_ls.append(joblib.load(fname))

In [55]:
# Create an empty dictionary to save residuals
bayes_mod_resiudal_dict = dict()

# Loop through the ts indexes. For each
for i in range(1, 77):
    # Create the y and X data frames
    y_train_sub = train_val_full.query("ts_index==@i").iloc[:,0:(lag_n+1)].dropna().iloc[:,0].copy()
    X_train_sub = train_val_full.query("ts_index==@i").iloc[:,0:(lag_n+1)].dropna().iloc[:,1:].copy()
    
    # Predict on the X data frame
    train_preds_sub = bayes_mod_ls[i-1].predict(X_train_sub)
    
    # Compute residuals and convery to list
    res_sub = (y_train_sub - train_preds_sub).to_list()
    
    # Add that list to the residual dictionary
    bayes_mod_resiudal_dict[i] = res_sub

In [56]:
# Create a dictionary for model performance
bayes_mod_test_preds = dict()

# Loop through the ts indexes
for i in range(1,77):
    # Create the X data frame to predict on 
    X_test_sub = test_df_full.query("ts_index==@i").iloc[:,1:(lag_n+1)]
    
    # Compute the test predictions
    test_pred_sub = bayes_mod_ls[i-1].predict(X_test_sub)
    
    # Save the predictions to the dictionary
    bayes_mod_test_preds[i] = test_pred_sub

In [57]:
# Define n_boot, the number of bootstramp samples to use for PI computation
n_boot = 1000

# Create a data frame to save PIs to
bayes_pred_int_df = pd.DataFrame()

# Set random seed for reproducibility
np.random.seed(54321)
random.seed(54321)

# Loop through the ts indexes in our list
for i in range(1,77):
    
    # Grab the true data y for that index
    y_test_sub = test_df_full.query("ts_index==@i").iloc[:,0]
    
    # Create empty lists to save PIs to
    percent_95_lo_ls = list()
    percent_95_hi_ls = list()
    percent_80_lo_ls = list()
    percent_80_hi_ls = list()
    
    for j in range(1344):
        # For each observation in the test set
        # Extract the predicted value
        pred = bayes_mod_test_preds[i][j]
        # Compute a bootstramp sample from the appropriate residual list
        resid_boot = np.random.choice(bayes_mod_resiudal_dict[i], size=n_boot, replace=True)
        # Add the prediction to the bootstrapped residual sample
        resid_preds = pred+resid_boot
        
        # From that, compute percentiles for the 95% and then 80% PIs and append those to lists
        percent_95_lo = np.percentile(resid_preds, 2.5)
        percent_95_hi = np.percentile(resid_preds, 97.5)
        percent_95_lo_ls.append(percent_95_lo)
        percent_95_hi_ls.append(percent_95_hi)
        
        percent_80_lo = np.percentile(resid_preds, 10)
        percent_80_hi = np.percentile(resid_preds, 90)
        percent_80_lo_ls.append(percent_80_lo)
        percent_80_hi_ls.append(percent_80_hi)
    
    # Create a temp data frame which includes the ts_index, true values, and PI for each observation
    pred_int_df_sub = pd.DataFrame({"ts_index": i,
                                    "actual": y_test_sub,
                                    "pct_95_lo": percent_95_lo_ls,
                                    "pct_95_hi": percent_95_hi_ls,                                   
                                    "pct_80_lo": percent_80_lo_ls,
                                    "pct_80_hi": percent_80_hi_ls
                                   })
    
    # Append that temp df to the full df
    bayes_pred_int_df = bayes_pred_int_df.append(pred_int_df_sub)

In [58]:
# Compute the PI scores for all observations for both 80% and 95% PIs
bayes_pred_int_df['int_80_score'] = interval_score(bayes_pred_int_df['actual'], 
                                                   bayes_pred_int_df['pct_80_lo'], 
                                                   bayes_pred_int_df['pct_80_hi'],
                                                   0.8
                                                  )

bayes_pred_int_df['int_95_score'] = interval_score(bayes_pred_int_df['actual'], 
                                                   bayes_pred_int_df['pct_95_lo'], 
                                                   bayes_pred_int_df['pct_95_hi'],
                                                   0.95
                                                  )

In [59]:
# Print the mean PI scores
bayes_pred_int_df.mean()

ts_index         38.500000
actual          265.435072
pct_95_lo       237.473989
pct_95_hi       295.911708
pct_80_lo       249.306684
pct_80_hi       283.604751
int_80_score    149.477249
int_95_score    393.408032
dtype: float64

In [60]:
# Group by time series index and compute the normalized interval scores
bayes_pred_int_df_grouped = bayes_pred_int_df.groupby("ts_index")\
.agg({"int_95_score":"mean", "int_80_score":"mean", "actual":"mean"}).reset_index()

bayes_pred_int_df_grouped['int_95_score_scaled'] = bayes_pred_int_df_grouped['int_95_score']/bayes_pred_int_df_grouped['actual']
bayes_pred_int_df_grouped['int_80_score_scaled'] = bayes_pred_int_df_grouped['int_80_score']/bayes_pred_int_df_grouped['actual']

In [61]:
# Print the average scaled interval scores
bayes_pred_int_df_grouped[['int_80_score_scaled', 'int_95_score_scaled']].mean()

int_80_score_scaled    0.642251
int_95_score_scaled    1.644127
dtype: float64

In [62]:
# Save the PI df to a csv
bayes_pred_int_df.to_csv("Results/Univariate/LightGBM Bayes/prediction_intervals.csv", index=False)

[LightGBM] [Warning] lambda_l1 is set=0.4728723294575521, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.4728723294575521
[LightGBM] [Warning] lambda_l2 is set=0.7295534508088319, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.7295534508088319
[LightGBM] [Warning] lambda_l1 is set=0.5198377424488568, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.5198377424488568
[LightGBM] [Warning] lambda_l2 is set=0.7148292420698871, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.7148292420698871
[LightGBM] [Warning] lambda_l1 is set=0.2334499219749726, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.2334499219749726
[LightGBM] [Warning] lambda_l2 is set=0.3629403893934763, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.3629403893934763
